In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dd

In [92]:
df = pd.read_csv('dataset.csv')

# ANALISE EXPLORATÓRIA

In [ ]:
df['Valor_Venda'].describe()

In [ ]:
df[df.duplicated()]

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
for i in df.columns:
  print(i)

PERGUNTA 1

In [ ]:
# Primeiro filtra a coluna
df_pl = df[df['Categoria'] == 'Office Supplies']
# Agora eu agrupo as cidades
df_pl_total = df_pl.groupby('Cidade')['Valor_Venda'].sum()
# Resultado
cd_maior_venda = df_pl_total.idxmax()
print(f'A cidade com mais vendas é {cd_maior_venda}')

PERGUNTA 2

In [ ]:
df_p2 = df.groupby('Data_Pedido')['Valor_Venda'].sum()

# Criação do grafico
plt.figure(figsize = (29, 10))
df_p2.plot(x = 'Data_Pedido', y = 'Valor_Venda', color='green')
plt.title('Total de vendas por pedido')
plt.show()

PERGUNTA 3

In [ ]:
df_p3 = df.groupby('Estado')['Valor_Venda'].sum()

# Criação do grafico
plt.figure(figsize = (10, 6))
df_p3.plot(x = 'Estado', y = 'Valor_Venda', color='blue')
plt.show()

PERGUNTA 4

In [ ]:
df_p4 = df.groupby('Cidade')['Valor_Venda'].sum().reset_index().sort_values(by = 'Valor_Venda',
                                                                            ascending = False).head(10)
df_p4.head(10)
plt.figure(figsize = (16, 6))
sns.set_palette('coolwarm')
sns.barplot(data = df_p4, x = 'Valor_Venda', y = 'Cidade').set(title = 'as 10 cidade com maior total de vendas')


PERGUNTA 5

In [ ]:
df_p5 = df.groupby('Segmento')['Valor_Venda'].sum().reset_index().sort_values(by='Valor_Venda', ascending=False)

def form(pct):
    total = sum(df_p5['Valor_Venda'])
    vel = int(round(pct * total / 100.0))
    return f'${vel}'

plt.figure(figsize=(16, 6))

plt.pie(df_p5['Valor_Venda'],
        labels=df_p5['Segmento'],
        autopct=form,
        startangle=90)

# Limpa o centro
centro = plt.Circle((0, 0), 0.82, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centro)

plt.annotate(text="Total de vendas: $" + str(int(sum(df_p5['Valor_Venda']))), xy=(-0.25, 0))
plt.title("Total de vendas por segmento")
plt.show()

PERGUNTA 6

In [ ]:
df['Data_Pedido'] = pd.to_datetime(df['Data_Pedido'], dayfirst=True)

# Extraindo o ano
df['Ano'] = df['Data_Pedido'].dt.year
df_p6 = df.groupby(['Ano', 'Segmento'])['Valor_Venda'].sum()
df_p6

PERGUNTA 7

In [ ]:
df['Desconto'] = np.where(df['Valor_Venda'] > 1000, 0.15, 0.10)
df['Desconto'].value_counts()
print('no total de 457 vendas receberiam 155')

In [ ]:
df.columns

PERGUNTA 8

In [ ]:
df['Valor_Venda_Desconto'] = df['Valor_Venda'] - (df['Valor_Venda'] * df['Desconto'])

# filtrando as vendas antes do desconto
df_p8_antes = df.loc[df['Desconto'] == 0.15, 'Valor_Venda']

# filtrando as vendas depois do desconto
df_p8_depois = df.loc[df['Desconto'] == 0.15, 'Valor_Venda_Desconto']

# media antes do desconto
media_antes = df_p8_antes.mean()

# media depois do desconto
media_depois = df_p8_depois.mean()

print("Media de desconto antes dos 15% : ", round(media_antes, 2))
print("Media de desconto depois dos 15% : ",round(media_depois, 2))

PERGUNTA 9

In [ ]:
df['Mes'] = df['Data_Pedido'].dt.month

# arupando por ano, mes e segmento
df_p9 = df.groupby(['Ano', 'Mes', 'Segmento'])['Valor_Venda'].agg([np.sum, np.mean, np.median])

# extraindo os niveis
anos =  df_p9.index.get_level_values(0)
meses =  df_p9.index.get_level_values(1)
segmento =  df_p9.index.get_level_values(2)

df.head()
# criando grafico
plt.figure(figsize = (12, 6))
sns.set()
fig1 = sns.relplot(kind = 'line',
                  data = df_p9,
                  y = 'mean',
                  x = meses,
                  hue = segmento,
                  col = anos,
                  col_wrap = 4)
plt.show()

In [ ]:
df.columns

PERGUNTA 10

In [ ]:
df_p10 = df.groupby(['Categoria',
                     'SubCategoria']).sum(numeric_only = True).sort_values('Valor_Venda',
                                                                           ascending = False).head(12)

df_p10 = df_p10[['Valor_Venda']].astype(int).sort_values(by = 'Categoria').reset_index()
df_p10_cat = df_p10.groupby('Categoria').sum(numeric_only = True).reset_index()

cores_cat = ['#6e7dd5', '#eb0318', '#eb0318']

cores_subcategorias = ['#4634eb', '#eb3459', '#eb34e8',
                       '#ebb134', '#d3eb34', '#34eb6b',
                       '#d1c3ba', '#0d0b0a', '#eb3434',
                       '#63d2d4', '#d463ab', '#d48d63']

fig, ax = plt.subplots(figsize = (18, 12))
# grafico categoria
pl = ax.pie(df_p10_cat ['Valor_Venda'],
            radius = 1,
            labels = df_p10_cat['Categoria'],
            wedgeprops = dict(edgecolor = 'white'),
            colors = cores_cat)

# grafico subcategoria
p2 = ax.pie(df_p10['Valor_Venda'],
            radius = 0.9,
            labels = df_p10['Categoria'],
            colors = cores_subcategorias,
            labeldistance = 0.7,
            wedgeprops = dict(edgecolor = 'white'),
            pctdistance = 0.53,
            rotatelabels = True)

# limpa o centro
centro = plt.Circle((0, 0), 0.6, fc = 'white')

fig = plt.gcf()
fig.gca().add_artist(centro)
plt.annotate(text = 'Total de vendas' + '$' + str(int(sum(df_p10['Valor_Venda']))), xy = (-0.2, 0))
plt.title('Total de vendas por categorias e Top 12 subcategorias')
plt.show()

In [93]:
df.to_csv('DesafioCompleto.csv')